In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from architectures.loaders import load_architecture
from architectures.lora import add_lora

from utils import get_args

args = get_args()
args.arch = 'LeNet5'
args.dataset = 'CIFAR10'
args.selection_method = 'none'
args.batch_size = 128

model, target_layers = load_architecture(args,)
# # add_lora(target_layers, model)

# # # Assuming you have your model, clean_loss_fn, and robust_loss_fn already defined

# import trades

# # # Assuming you have your data loaders for clean and adversarial examples
# from datasets import IndexedDataset
# dataset =  IndexedDataset(args, train=True)

# data_loader = torch.utils.data.DataLoader(
#         dataset,
#         batch_size=args.batch_size,
#         shuffle=False,
#     )

In [14]:
from tqdm.notebook import tqdm
optimizer = optim.SGD(model.parameters(), lr=0.01)

def compute_gradients(data_loader,loss_type, optimizer):

    model.train()
    optimizer.zero_grad()
    
    total_gradients = None
    num_samples = 0

    # data_iter = iter(data_loader)
    # data, target, idx = next(data_iter)
    for data, target, idx in tqdm(data_loader):

        optimizer.zero_grad()
            
        data, target = data.to('cuda'), target.to('cuda')  # Assuming you're using GPU
                
        loss_natural, loss_robust, loss = trades.trades_loss(model=model, x_natural=data, y=target, optimizer=optimizer,)
                
        if loss_type == 'global':
            loss_robust.backward()  # Compute gradients
        elif loss_type == 'clean':
            loss_natural.backward()  # Compute gradients
        elif loss_type == 'robust':
            loss.backward()  # Compute gradients
        else:
            print('error')
        
        # Accumulate gradients
        if total_gradients is None:
            total_gradients = [param.grad.clone().detach() for param in model.parameters() if param.grad is not None]
        else:
            for i, param in enumerate(model.parameters()):
                if param.grad is not None:
                    total_gradients[i] += param.grad.clone().detach()
            
        num_samples += data.size(0)
        
    # Average gradients over the batch
    for i in range(len(total_gradients)):
        total_gradients[i] /= num_samples
    
    return total_gradients

def cosine_similarity(grad1, grad2):
    # Flatten gradients into vectors
    grad1_vector = torch.cat([g.view(-1) for g in grad1])
    grad2_vector = torch.cat([g.view(-1) for g in grad2])
    
    cos_sim = nn.functional.cosine_similarity(grad1_vector, grad2_vector, dim=0)
    
    return cos_sim.item()

# Compute gradients for clean loss
clean_gradients = compute_gradients(data_loader, 'clean', optimizer)

# # Compute gradients for robust loss
# robust_gradients = compute_gradients(data_loader, 'robust')

# global_gradients = compute_gradients(data_loader, 'global')

# # Calculate cosine similarity between the clean and robust gradients
# gradient_alignment = cosine_similarity(clean_gradients, robust_gradients)

# print(f"Cosine Similarity between Clean and Robust Gradients: {gradient_alignment:.4f}")


  0%|          | 0/782 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.94 GiB of which 4.12 MiB is free. Including non-PyTorch memory, this process has 2.93 GiB memory in use. Of the allocated memory 2.79 GiB is allocated by PyTorch, and 26.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
gradient_alignment = cosine_similarity(clean_gradients, clean_gradients)
print(gradient_alignment)
# gradient_alignment = cosine_similarity(robust_gradients, robust_gradients)
# print(gradient_alignment)
# gradient_alignment = cosine_similarity(clean_gradients, robust_gradients)
# print(gradient_alignment)

1.0
0.9999998807907104
-0.20789965987205505
